# Week 4: The geometry of meaning

We're going to explore some basic forms of text analysis, using David Robinson's dataset of tweets made from the account of Donald J. Trump, as well as a dataset of nineteenth-century poetry and fiction, which is divided by date, by genre, and also by reception (whether or not the volume got reviewed in an 'elite' journal).

To begin, let's import some modules we're going to need later, and also read in the Trump data.

In [2]:
import os, csv, math
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

cwd = os.getcwd()
print('Current working directory: ' + cwd + '\n')
      
relativepath = os.path.join('..', 'data', 'weekfour', 'trump.csv')
trump = pd.read_csv(relativepath)

Current working directory: /Users/rdubnic2/Documents/lis590dsh/Code



## Different ways of identifying "distinctive" words

In this section we'll explore Dunning's log-likelihood, and also think about the strengths and weaknesses of "distinctive" words as evidence.

First let's glance at the Trump dataset.

In [3]:
trump.head()

,Unnamed: 0,Unnamed: 0.1,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,0,1,My economic policy speech will be carried live...,False,9214,NaN,2016-08-08 15:20:44,False,NaN,762669882571980801,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,3107,False,False,NaN,NaN
1,1,2,"Join me in Fayetteville, North Carolina tomorr...",False,6981,NaN,2016-08-08 13:28:20,False,NaN,762641595439190016,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,2390,False,False,NaN,NaN
2,2,3,"#ICYMI: ""Will Media Apologize to Trump?"" https...",False,15724,NaN,2016-08-08 00:05:54,False,NaN,762439658911338496,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,6691,False,False,NaN,NaN
3,3,4,"Michael Morell, the lightweight former Acting ...",False,19837,NaN,2016-08-07 23:09:08,False,NaN,762425371874557952,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,6402,False,False,NaN,NaN
4,4,5,The media is going crazy. They totally distort...,False,34051,NaN,2016-08-07 21:31:46,False,NaN,762400869858115588,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,11717,False,False,NaN,NaN


#### Basic functions

For a lot of the work we do today, we're going to want to construct dictionaries that hold the frequencies of words in different categories: poetry or fiction, Trump-iphone or Trump-android. To do this we'll need to break text into words, count the words in each text, and then add up the counts by category.

Let's define some functions that do this. (You can find more polished versions of these functions in the ```nltk``` module.)


In [4]:
def tokenize(astring):
    ''' Breaks a string into words, and counts them.
    Designed so it strips punctuation and lowercases everything,
    but doesn't separate hashtags and at-signs.
    '''
    wordcounts = Counter()
    # create a counter to hold the counts
    
    tokens = astring.split()
    for t in tokens:
        word = t.strip(',.!?:;-—()<>[]/"\'').lower()
        wordcounts[word] += 1
        
    return wordcounts

def addcounters(counter2add, countersum):
    ''' Adds all the counts in counter2add to countersum.
    Because Counters(like dictionaries) are mutable, it
    doesn't need to return anything.
    '''
    
    for key, value in counter2add.items():
        countersum[key] += value

def create_vocab(seq_of_strings, n):
    ''' Given a sequence of text snippets, this function
    returns the n most common words. We'll use this to
    create a limited 'vocabulary'.
    '''
    vocab = Counter()
    for astring in seq_of_strings:
        counts = tokenize(astring)
        addcounters(counts, vocab)
    topn = [x[0] for x in vocab.most_common(n)]
    return topn

# Let's test the vocabulary function.
vocab = create_vocab(trump['text'], 4000)
vocab[0:10]
        

['the', 'to', 'and', 'a', 'in', 'is', 'i', 'you', 'of', 'will']

#### A few more basic functions

Once we have a vocabulary, we're going to want to divide our texts into categories, create Counters summing the word frequencies in those categories, and then compare the two Counters to find words that are overrepresented in one category relative to the other.

There are several ways we could define "overrepresented." We'll use Robinson's simple log-odds measure, as well as Dunning's log-likelihood.

In [5]:
def logodds(countsA, countsB, word):
    ''' Straightforward.
    '''
    
    odds = (countsA[word] + 1) / (countsB[word] + 1)
    
    # Why do we add 1 on both sides? Two reasons. The hacky one is 
    # that otherwise we'll get a division-by-zero error whenever
    # word isn't present in countsB. The more principled reason
    # is that this technique (called Laplacian smoothing) tends
    # to reduce the dramatic disproportion likely to be found in
    # very rare words.
    
    return math.log(odds)

def signed_dunnings(countsA, totalA, countsB, totalB, word):
    ''' Less straightforward. This function calculates a signed (+1 / -1)
    version of Dunning's log likelihood. Intuitively, this is a number 
    that gets larger as the frequency of the word in our two corpora
    diverges from its EXPECTED frequency -- i.e., the frequency it would
    have if it were equally distributed over both. But it also tends to get
    larger as the raw frequency of the word increases.
    
    Note that this function requires two additional arguments:
    the total number of words in A and B. We could calculate that inside
    the function, but it's faster to calculate it just once, outside the function.
    
    Also note: the strict definition of Dunnings has no 'sign': it gets bigger
    whether a word is overrepresented in A or B. I've edited that so that Dunnings
    is positive if overrepresented in A, and negative if overrepresented in B.
    '''
    if word not in countsA and word not in countsB:
        return 0
    
    # the raw frequencies of this word in our two corpora
    # still doing a little Laplacian smoothing here
    a = countsA[word] + 0.1
    b = countsB[word] + 0.1
    
    # now let's calculate the expected number of times this
    # word would occur in both if the frequency were constant
    # across both
    overallfreq = (a + b) / (totalA + totalB)
    expectedA = totalA * overallfreq
    expectedB = totalB * overallfreq
    
    # and now the Dunning's formula
    dunning = 2 * ((a * math.log(a / expectedA)) + (b * math.log(b / expectedB)))
    
    if a < expectedA:
        return -dunning
    else:   
        return dunning

# a set of common words is often useful
stopwords = {'a', 'an', 'are', 'and', 'but', 'or', 'that', 'this', 'so', 
             'all', 'at', 'if', 'in', 'i', 'is', 'was', 'by', 'of', 'to', 
             'the', 'be', 'you', 'were'}

# finally, one more function: given a list of tuples like
testlist = [(10, 'ten'), (2000, 'two thousand'), (0, 'zero'), (-1, 'neg one'), (8, 'eight')]
# we're going to want to sort them and print the top n and bottom n

def headandtail(tuplelist, n):
    tuplelist.sort(reverse = True)
    print("TOP VALUES:")
    for i in range(n):
        print(tuplelist[i][1], tuplelist[i][0])
    
    print()
    print("BOTTOM VALUES:")
    lastindex = len(tuplelist) - 1
    for i in range(lastindex, lastindex - n, -1):
        print(tuplelist[i][1], tuplelist[i][0])
        
headandtail(testlist, 2)
    

TOP VALUES:
two thousand 2000
ten 10

BOTTOM VALUES:
neg one -1
zero 0


## Exercise 1: Is Dunning's a better measure than logodds for Trump's tweets?

Let's put all these functions together to answer that question.

I've sketched the outline of a program below in "pseudocode," which
describes what needs to be done. Translate that into real Python code, using
the functions defined above. First use Robinson's logodds function and try to
replicate his results. See what happens if you do (or don't) remove stopwords
and tweets that begin with a quote.
                                                   
Then edit your code to use Dunning's log likelihood. Does that seem to be a better (more revealing) measure of overrepresentation? How would we decide?

In [6]:
# Code for Exercise 1

# Start by creating a vocabulary for words in the Trump tweets.
# Put it in a variable called 'vocab'.

vocab = create_vocab(trump['text'], 5000)
print('Total vocab = ', len(vocab))

# Remember the function create_vocab takes two arguments:
# (seq_of_strings, n)
# We can afford to include all the words, so set n for 5000.

# An optional step: removing stopwords
# vocab = list(set(vocab) - stopwords)

vocab_nostop = list(set(vocab) - stopwords)
print('Total vocab minus stop words = ', len(vocab_nostop))

# Create counters for the android and iphone corpora.

android = Counter()
iphone = Counter()

num_rows = trump.shape[0]
print('Total number of rows in Trump dataset = ', num_rows)

# Figure out how many rows are in the Trump DataFrame
# and put that number in a variable like 'numrows.'
# Then iterate through the 'text' column of the data frame.

for i in range(num_rows):
    counts = tokenize(trump['text'][i])
    if 'iphone' in trump['statusSource'][i]:
        addcounters(counts, iphone)
    elif 'android' in trump['statusSource'][i]:
        addcounters(counts, android)

android_count = len(android)
iphone_count = len(iphone)

print('Total android words = ', android_count)
print('Total iphone words = ', iphone_count)


# for each text cell, get a Counter with words counts for that cell
# then add those counts either to iphone or android, like so:

#     if 'iphone' in trump['statusSource'][i]:
#         addcounters(counts, iphone)
#     elif 'android' in trump['statusSource'][i]:
#         addcounters(counts, android)

# When you get around to running Dunning's, you'll need to
# create variables that hold the total count of *all words*
# in iphone and android.

total_words = Counter()
addcounters(android, total_words)
addcounters(iphone, total_words)

print('Total word count = ', len(total_words))

# Create an empty list to hold pairs of (overrepresentation_measure, word)
# Then iterate through your vocabulary. For each word, measure 
# overrepresentation using either logodds or signed_dunnings.
# Create a tuple, (overrepresentation_measure, word)
# and append it to the empty list you created.

overrep_words_log = []
overrep_words_dunnings = []

for wrd in vocab_nostop:
    x = signed_dunnings(iphone, iphone_count, android, android_count, wrd)
    overrep_words_dunnings.append((x, wrd))

for word in vocab:
    x = logodds(iphone, android, word)
    overrep_words_log.append((x, word))

# Finally use the headandtail function to display the top 25 and bottom 25
# words in your tuplelist.

print('\n','FOR LOG ODDS:')
headandtail(overrep_words_log, 25)

print('\n', 'FOR DUNNINGS:')
headandtail(overrep_words_dunnings, 25)

Total vocab =  4682
Total vocab minus stop words =  4659
Total number of rows in Trump dataset =  1512
Total android words =  2975
Total iphone words =  2535
Total word count =  4448

 FOR LOG ODDS:
TOP VALUES:
#imwithyou 3.044522437723423
#trumppence16 2.70805020110221
join 2.662587827025453
#americafirst 2.6390573296152584
#trump2016 2.5084371471981943
video 2.4849066497880004
7pm 2.4849066497880004
#votetrump 2.4849066497880004
#rncincle 2.3978952727983707
#inprimary 2.3978952727983707
#wiprimary 2.1972245773362196
#fitn 2.1972245773362196
#crookedhillary 2.1972245773362196
officers 2.0794415416798357
3pm 2.0794415416798357
tomorrow 2.03688192726104
tickets 2.0149030205422647
refugees 1.9459101490553132
#votetrumpnh 1.9459101490553132
â 1.791759469228055
greatly 1.791759469228055
daytona 1.791759469228055
choker 1.791759469228055
7pme 1.791759469228055
#gopdebate 1.791759469228055

BOTTOM VALUES:
@realdonaldtrump -4.127134385045092
a.m -2.70805020110221
@megynkelly -2.6390573296152

## Exercise 2: Apply the same methods to a more literary dataset.

I've also provided a dataset of roughly 1026 snippets from nineteenth-century poetry and fiction. The code below should read it in. Run that, then copy and paste the code you worked up for Trump, and edit it so it provides the most distinctive words for poetry (versus fiction).

If we have time, it may also be worth distinguishing poetry reviewed in elite journals from poetry that wasn't.


In [7]:
relativepath = os.path.join('..', 'data', 'weekfour', 'poefic.csv')
poefic = pd.read_csv(relativepath)
poefic.head()

,date,author,title,genre,reception,text
0,1908,"Robins, Elizabeth,",The convert,fiction,elite,"looked like decent artisans, but more who bore..."
1,1871,"Lytton, Edward Bulwer Lytton,",The coming race,fiction,elite,"called the "" Easy Time "" (with which what I ma..."
2,1872,"Butler, Samuel,","Erewhon, or, Over the range",fiction,elite,the curtain ; on this I let it drop and retrea...
3,1900,"Barrie, J. M.",Tommy and Grizel,fiction,elite,"at you !"" he said. ""Dear eyes, "" said she. ""Th..."
4,1873,"Ritchie, Anne Thackeray,",Old Kensington,fiction,elite,"furious; I have not dared tell her, poor creat..."


In [8]:
# Code for Exercise 2

# The main thing you will need to change is the code that
# identifies rows as belonging to one of two corpora.


## Using corpora to create a "meaning space."

Contrasting two corpora can be revealing, but sometimes we want to think about the relations between individual texts or words. To do that, we often represent them as vectors in a multi-dimensional space.

The simplest way to do this is to create a DataFrame where rows are documents and columns are word — a document-term matrix. Here's a function that does that. It requires a pre-defined vocabulary (list of words) as well as a list (or numpy vector) of texts.


In [9]:
def doc_term_matrix(vocab, textvector):
    ''' Transform the textvector into a document-term matrix
    with one column for each word in vocab.
    '''
    
    n = len(textvector)
    vocabset = set(vocab)
    # making a set so we can check membership quickly;
    # it's much faster in a set than in a list
    
    termdictionary = dict()
    for word in vocab:
        termdictionary[word] = np.zeros(n)
    for i, text in enumerate(textvector):
        counts = tokenize(text)
        for word, count in counts.items():
            if word in vocabset:
                termdictionary[word][i] += count
    
    dtmatrix = pd.DataFrame(termdictionary, columns = vocab)
    return dtmatrix

# A nice arcane trick to perform on a document-term matrix
# is to squash it into a smaller number of dimensions. This
# often reveals relationships between words that don't
# necessarily, literally occur together. The technique is called
# Latent Semantic Analysis.

def lsa_matrix(dtmatrix, vocab, number_of_dimensions):
    lsa = TruncatedSVD(number_of_dimensions, algorithm = 'arpack')
    dtm_lsa = lsa.fit_transform(dtmatrix)
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
    lsamatrix = pd.DataFrame(lsa.components_, columns = vocab)
    
    return lsamatrix

def cosine_similarity(vector1, vector2):
    dot_product = np.sum(vector1 * vector2)
    # we assume these are numpy vectors and can be
    # multiplied elementwise
    
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    else:
        return dot_product/magnitude

## Exercise 3: Finding words that are close in "meaning space."

Following Widdows, we'll measure semantic similarity as the cosine similarity between vectors defined by a word's distribution across documents.

Let's try this both in the space defined by Trump tweets and in the space defined by 19c literature.

In [ ]:
# Code for exercise 3

# Let's start by getting a vocabulary,
# and a doc-term matrix, as well as a
# squashed (lsa) version of that matrix.

vocab = create_vocab(trump['text'], 5000)
dtm = doc_term_matrix(vocab, trump['text'])
lsa = lsa_matrix(dtm, vocab, 25)

# Now write a function called find_matches
# that prints the 10 closest, and
# 10 weakest, matches for a given word in a given matrix
# Your function should take a matrix and a word as
# arguments.

def find_matches(amatrix, word):
    colA = amatrix[word]
    vocab = amatrix.columns.values
    tupelist = []
    for w in vocab:
        colB = amatrix[w]
        cosinesim = cosine_similarity(colA, colB)
        tupelist.append((cosinesim, w))
    
    headandtail(tupelist, 10)

# It can get a vocabulary from the columns
# vocab = amatrix.columns.values

# Then use the word to get a vector associated with
# that column.

# Create a list to hold tuples, as we did before.

# Iterate through all columns, and for
# each column, check the cosine similarity between
# that column and our word. In each case, add
# (cosine similarity, word) to the tuple list.

# Then use headandtail(tuplelist, n) to display the
# top and bottom closest matches.

# Once you've defined this function (and run this cell),
# the cell below should allow you to select words
# and find matches. You can start by looking for
# matches in the doc-term matrix, but then
# branch out to the lsa matrix. See if that's
# better or worse.

# Then apply the same technique to the poefic DataFrame

In [1]:
user_word = input('word? ')
find_matches(dtm, user_word)

word? hillary


NameError: name 'find_matches' is not defined